In [0]:
# Databricks widgets are used to parameterize the notebook, allowing users to specify values at runtime.
# These widgets make the notebook reusable and interactive for different catalogs, schemas, volumes, and document URLs.
dbutils.widgets.text("catalog_use", "dev")	# Catalog name
# Widget for specifying the schema to use
# Default is 'ncpdp_rx'
dbutils.widgets.text("schema_use", "ncpdp_rx")	# Schema name
# Widget for specifying the volume to use
# Default is 'spec_documents'
dbutils.widgets.text("volume_use", "spec_documents")	# Volume name
# Widget for specifying an optional subdirectory within the volume
# Default is empty string (no subdirectory)
dbutils.widgets.text("volume_sub_path", "")	# Optional subdirectory
# Widget for specifying the URL of the document to download
# Default is a sample NCPDP PDF
dbutils.widgets.text("spec_document_url", "https://ncpdp.org/NCPDP/media/pdf/Payer_Sheet_Template_1.pdf")

In [0]:
# Extract widget values and strip whitespace for safety
# These variables will be used throughout the notebook
catalog_use = dbutils.widgets.get("catalog_use").strip()
schema_use = dbutils.widgets.get("schema_use").strip()
volume_use = dbutils.widgets.get("volume_use").strip()
volume_sub_path = dbutils.widgets.get("volume_sub_path").strip()
spec_document_url = dbutils.widgets.get("spec_document_url").strip()

In [0]:
# Print the extracted widget values for verification and debugging
# This helps ensure the correct parameters are being used
print(
f"""
catalog_use: {catalog_use}
schema_use: {schema_use}
volume_use: {volume_use}
volume_sub_path: {volume_sub_path}
spec_document_url: {spec_document_url}
"""
)

In [0]:
# Construct the volume path based on widget values
# If volume_sub_path is empty, use the base volume path
# Otherwise, append the sub-path
# Check if volume_sub_path is None or an empty string
if volume_sub_path is None \
		or volume_sub_path == "":
	# Construct the volume path without the sub-path
	volume_path = f"/Volumes/{catalog_use}/{schema_use}/{volume_use}"
else:
	# Construct the volume path with the sub-path
	volume_path = f"/Volumes/{catalog_use}/{schema_use}/{volume_use}/{volume_sub_path}"

# Print the constructed volume path for confirmation
print(f"""
	volume_path: {volume_path}
""")

In [0]:
import urllib.request
import os

os.makedirs(volume_path, exist_ok=True)

# Function to download a file from a URL and save it to the specified volume path
# Allows overwrites and sets file permissions so any user with access to the volume can read or overwrite
# Args:
#   url (str): The URL of the file to download
#   volume_path (str): The Unity Catalog volume path to save the file
#   filename (str): The name to save the file as
def download_file_to_volume(url: str, volume_path: str, filename: str):
	"""
	Download a file from a URL and save it to the specified volume path.
	Allows overwrites and sets file permissions so any user with access to the volume can read or overwrite.
	Args:
		url (str): The URL of the file to download.
		volume_path (str): The Unity Catalog volume path to save the file.
		filename (str): The name to save the file as.
	"""
	file_path = f"{volume_path}/{filename}"
	# Download and overwrite if exists
	urllib.request.urlretrieve(url, file_path)
	# Set permissions: read/write for all users
	os.chmod(file_path, 0o666)
	return file_path

In [0]:
# Download the specified document from the URL into the Unity Catalog volume
# The filename is set explicitly for clarity and consistency
file_path = download_file_to_volume(
    url=spec_document_url
    ,volume_path=volume_path
    ,filename="Payer_Sheet_Template_1.pdf"
)

In [0]:
# List the contents of the downloaded file's location to confirm the file was saved successfully
# This displays the file and its metadata in the notebook
display(
    dbutils.fs.ls(file_path)
)